## Project sentiment stock

### Objective:
- In this project, we will perform a sentiment analysis of data published on X (Twitter), in real time, relating to stock market quotes. This analysis can be passed on to our investors, through sentiment percentages, whether the messages about the quotes present a positive or negative trend for the subject in question.

- Nesse projeto vamos fazer uma análise de sentimento de dados publicados no X (twitter), em tempo real, relativo às cotações da bolsa de valores. Essa análise vai poder passar para nossos investidores, através dos percentuais de sentimento, se as postagens sobre as cotações, apresentam uma tendência positiva ou negativa para o assunto tratado.

### Data Origin:
- Dataset: financial_tweets.csv

- This file was acquired by capturing tweets from users who made publications within the subject discussed here, expressing their feelings about stock market quotes in real time. The data is intercepted through an API provided by X (twitter) and saved in a .csv file.

- Esse arquivo foi adquirido através da captura de tweets dos usuários que fizeram publicações dentro do assunto aqui tratado, expressando seus sentimentos sobre cotações da bolsa em tempo real. Os dados são interceptados através de API fornecida pelo X (twitter) e gravados em arquivo .csv.

Aqui o que nos interessa é somente o campo:
- Description: descrição do sentimento publicado pelo usuário.

- ## Análise exploratória dos dados
- ## Preparação dos dados
- ## Armazenamento dos dados tratados
- ## Geração de sentimento
    - textblob
    - vader
    - bert

In [2]:
# maximiza nro de linhas e colunas para exibição
# inibe mensagens de warning
import pandas as pd
pd.set_option('display.max_rows', None) # permite a máxima visualização das linhas em um display
pd.set_option('display.max_columns', None) # permite a máxima visualização das colunas em um display
import warnings
warnings.simplefilter('ignore') # inibe a exibição de avisos de warning

In [ ]:
# bibliotecas
import tweepy as tw # coleta Tweets em tempo real, permitindo que os desenvolvedores especifiquem filtros para buscar informações específicas.
import numpy as np # computação de matrizes

import re # realiza operações com expressões regulares, permitindo correspondência de padrões em strings.
import string # manipulação de strings, como a classe Template que implementa regras específicas para formatação.
from nltk.corpus import stopwords # possui uma lista de stopwords armazenada em 16 idiomas diferentes, facilitando a remoção de palavras irrelevantes em análises de texto.
from nltk.stem import PorterStemmer # algoritmo de stemming que remove afixos morfológicos de palavras, deixando apenas a raiz da palavra.
from nltk.tokenize import TweetTokenizer # ferramenta projetada para dividir textos de redes sociais em tokens relevantes, especialmente preparada para lidar com a linguagem e abreviações comuns do Twitter.

from textblob import TextBlob # biblioteca Python voltada para o processamento de dados textuais, oferecendo uma API simples para tarefas comuns de processamento de linguagem natural (NLP).

import nltk # biblioteca em Python projetada para o processamento de linguagem natural (NLP). Ela oferece diversas ferramentas e recursos para facilitar a análise e manipulação de textos.
nltk.download('vader_lexicon') # abre uma interface que permite selecionar e baixar corpora, modelos e outros pacotes de dados específicos.

from nltk.sentiment.vader import SentimentIntensityAnalyzer # ferramenta de análise de sentimentos desenvolvida pela biblioteca NLTK, que utiliza um modelo baseado em regras para classificar textos informais em sentimentos positivos, negativos ou neutros.

from transformers import AutoModelForSequenceClassification # classe que fornece uma maneira rápida de ajustar um modelo de linguagem pré-treinado para uma tarefa de classificação de texto.
from transformers import AutoTokenizer # classe genérica de tokenização que é instanciada como uma das classes de tokenização da biblioteca quando criada com AutoTokenizer.from_pretrained().
import torch # framework de computação científica com suporte extensivo para algoritmos de aprendizado de máquina.
             # realizar cálculos numéricos de forma eficiente, especialmente operações com matrizes e tensores.

import tensorflow as tf # frameworks de aprendizado de máquina mais populares e amplamente utilizados no mundo.

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\KR\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
# chave e token definidas e associadas ao app criado no X (twitter) para captura de tweets com API
# https://developer.x.com/en/portal/dashboard
# as chaves e tokens abaixo são simbólicas devendo ser geradas por um usuário do X
consumer_key = 'bokA1eIxmcK5y396klmyThyxE'
consumer_secret = 'kpI1wT4So74kl6OVbv10rMUk5xkPNJZhkCCPScf5cyzHjLCJ62'
access_token = '2689849668-kaDvfg15xpeF4a1D6pL9ck1EnYTQhBjGklcS77S'
access_token_secret = 'zPdg70A5cnkmmXmuvM62TlkxoHl4IHJGpGfD3c5stfday'

In [ ]:
# cria uma variável(instância) para autenticação no X(twitter)
autorizacao = tw.OAuthHandler(consumer_key, consumer_secret)

In [ ]:
# autenticação para autorização do acesso à API do X (twitter)
autorizacao.set_access_token(access_token, access_token_secret)

In [ ]:
# cria uma variável instanciando a api do twitter com a autenticação feita anteriormente
api = tw.API(autorizacao)

In [ ]:
# retorna uma coleção dos Tweets e Retweets mais recentes postados pelo usuário autenticado e pelos usuários que ele segue.
meus_tweets = api.home_timeline()

In [ ]:
# exibe os tweets capturados
for twitters in meus_tweets:
    print(twitters.created_at)
    print(twitters.text,  '\n')

In [ ]:
# filtra somente os tweets necessários
query = "bitcoin AND price"
pesquisa = api.search(q=query,lang='en',tweet_mode="extended",count=50)

In [ ]:
# cria lista de tweets a partir da pesquisa
resultado = []

for twitters in pesquisa:
  resultado.append(twitters.created_at)
  resultado.append(twitters.full_text)

In [ ]:
# cria matriz com a lista de tweets
matriz_np = np.array(resultado)
matriz_ajustada = np.reshape(matriz_np, (50,2))

In [ ]:
# cria um dataframe a partir da matriz_ajustada
df = pd.DataFrame()

colunas = [
    'Data_publicacao',
    'tweet'
]

df = pd.DataFrame(matriz_ajustada,columns=colunas)

In [ ]:
# salva o dataframe com tweets em arquivo .csv
arquivo_tweets = 'dataset/financial_tweets.csv'
df.to_csv(arquivo_tweets, encoding='utf-8', index=False,quotechar='|')

In [ ]:
# carregando arquivo tratado em dataframe para não repetir a captura de tweets
df_sentiment = pd.read_csv('dataset/financial_tweets.csv', sep=',')

In [ ]:
# cria o dataframe df
df = df_sentiment

In [ ]:
# seleciona somente parte dos tweets
df = df[:2000]

In [ ]:
# verifica quantidade de linhas e colunas(variáveis) do dataframe
df.shape

(2000, 13)

In [ ]:
# selecionando somente a coluna descrição ou texto do twitter.
df = df['description']

In [ ]:
# verificando as primeiras linhas
df.head(5)

0    Crazy that $PRIME is going to >$1B market cap ...
1    Crazy part is, a lot of what I have been tradi...
2                       $AVAX\n\nStacking on support..
3                                        $SOL\n\nLFG!!
4    $INJ - Twitter been super bullish on this one ...
Name: description, dtype: object

In [ ]:
# cria variável de stopwords em inglês
# cria variável de caracteres de pontuação padrão
english_stopwords = stopwords.words("english")
punctuation_words = string.punctuation

In [ ]:
# cria uma lista de tweets a partir do dataframe
tweet_list = df.tolist()

In [ ]:
# substitui ocorrências de um padrão especificado por uma nova string dentro da descriçaõ do tweet
clean_tweets = []

for tweet in tweet_list:
    
    tweet = str(tweet) 

    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub(r'\$\w*', '', tweet)
    tweet = re.sub(r'$', '', tweet)
    tweet = re.sub(r'https?://[^\s\n\r]+', '', tweet)
    tweet = re.sub(r'https?://[A-Za-z0-9./]+','',tweet)
    tweet = re.sub(r'https//[A-Za-z0-9./]+','',tweet)
    tweet = tweet.replace("\n","")
    tweet = re.sub('@[\w]+', '', tweet)

    tweet = re.sub(r"n't", " not ", tweet)
    tweet = re.sub(r"\'s", " ", tweet)
    tweet = re.sub(r"\'ve", " have ", tweet)
    tweet = re.sub(r"\'re", " are ", tweet)
    tweet = re.sub(r"\'d", " would ", tweet)
    tweet = re.sub(r"\'ll", " will ", tweet) 

    tweet = tweet.lower()
    tweet = tweet.strip()

    clean_tweets.append(tweet)

In [ ]:
# exibe algumas descrições modificadas
clean_tweets[:4]

['crazy that  is going to > market cap at the next cycle top as the “  of this cycle” and you still don’t own any anon. -100 exit or bust bitch.',
 'crazy part is, a lot of what i have been trading is still up..',
 'stacking on support..',
 'lfg!!']

In [ ]:
# converte a lista de tweets em dataframe
df = pd.DataFrame(clean_tweets)

In [ ]:
# verifica as primeiras linhas do dataframe
df.head(5)

,0
0,crazy that is going to > market cap at the ne...
1,"crazy part is, a lot of what i have been tradi..."
2,stacking on support..
3,lfg!!
4,- twitter been super bullish on this one and t...


In [ ]:
# cria novo dataframe
df_tblob = df.copy()

In [ ]:
# cria novo dataframe
df_vader = df.copy()

In [ ]:
# cria novo dataframe
df_bert = df.copy()

In [ ]:
# verifica as primeiras linhas
df_tblob.head(5)

,0
0,crazy that is going to > market cap at the ne...
1,"crazy part is, a lot of what i have been tradi..."
2,stacking on support..
3,lfg!!
4,- twitter been super bullish on this one and t...


In [ ]:
# categoriza o sentimento com textblob salvando no dataframe
for row in df_tblob.itertuples():
    tweet = df_tblob.at[row[0],0]
    analysis = TextBlob(tweet)
    df_tblob.at[row[0], 'polarity'] = analysis.sentiment[0]
    df_tblob.at[row[0], 'subjectivity'] = analysis.sentiment[1]
    if analysis.sentiment[0]>0:
        df_tblob.at[row[0], 'Sentiment'] = "Positivo"
    elif analysis.sentiment[0]<0:
        df_tblob.at[row[0], 'Sentiment'] = "Negativo"
    else:
        df_tblob.at[row[0], 'Sentiment'] = "Neutro"

In [ ]:
# verifica as primeiras linhas com as novas colunas (variáveis)
df_tblob.head(5)

,0,polarity,subjectivity,Sentiment
0,crazy that is going to > market cap at the ne...,0.125000,0.600,Positivo
1,"crazy part is, a lot of what i have been tradi...",-0.600000,0.900,Negativo
2,stacking on support..,0.000000,0.000,Neutro
3,lfg!!,0.000000,0.000,Neutro
4,- twitter been super bullish on this one and t...,0.195833,0.325,Positivo


In [ ]:
# categoriza o sentimento com sentiment.vader salvando no dataframe
for index, row in df_vader[0].items():
    score = SentimentIntensityAnalyzer().polarity_scores(row)
    if score['neg'] > score['pos']:
        df_vader.loc[index, "Sentiment"] = "Negativo"
    elif score['pos'] > score['neg']:
        df_vader.loc[index, "Sentiment"] = "Positivo"
    else:
        df_vader.loc[index, "Sentiment"] = "Neutro"
        
    df_vader.loc[index, 'neg'] = score['neg']
    df_vader.loc[index, 'neu'] = score['neu']
    df_vader.loc[index, 'pos'] = score['pos']
    df_vader.loc[index, 'compound'] = score['compound']

In [ ]:
# verifica as primeiras linhas com as novas colunas (variáveis)
df_vader.head(5)

,0,Sentiment,neg,neu,pos,compound
0,crazy that is going to > market cap at the ne...,Negativo,0.182,0.765,0.053,-0.6597
1,"crazy part is, a lot of what i have been tradi...",Negativo,0.179,0.821,0.000,-0.3400
2,stacking on support..,Neutro,0.000,1.000,0.000,0.0000
3,lfg!!,Neutro,0.000,1.000,0.000,0.0000
4,- twitter been super bullish on this one and t...,Positivo,0.000,0.809,0.191,0.7650


In [ ]:
# carrega o pacote tokenizer nlptown/bert-base-multilingual-uncased-sentiment em variável
# instancia a classe de modelo de classificação de sequências em variável model
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [ ]:
# define função utilizando modelo de classificação de sequências (tokens)
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [ ]:
# verifica linhas e colunas
df_bert.shape

(2000, 1)

In [ ]:
# verifica as primeiras linhas
df_bert.head(5)

,0
0,crazy that is going to > market cap at the ne...
1,"crazy part is, a lot of what i have been tradi..."
2,stacking on support..
3,lfg!!
4,- twitter been super bullish on this one and t...


In [ ]:
# transforma os tweets em uma string
df_bert[0] = df_bert[0].astype("str")

In [ ]:
# categoriza o sentimento com BERT salvando no dataframe
for index, row  in df_bert[0].items():
    score = sentiment_score(row)
    if score > 3:
        df_bert.loc[index, "Sentiment"] = "Positivo"
    elif score < 3:
        df_bert.loc[index, "Sentiment"] = "Negativo"
    else:
        df_bert.loc[index, "Sentiment"] = "Neutro"

In [ ]:
# verifica as primeiras linhas
df_bert.head(5)

,0,Sentiment
0,crazy that is going to > market cap at the ne...,Negativo
1,"crazy part is, a lot of what i have been tradi...",Neutro
2,stacking on support..,Neutro
3,lfg!!,Negativo
4,- twitter been super bullish on this one and t...,Positivo


In [ ]:
# salva os dados tratados e criados em arquivo csv organizado pelo índice
df_tblob.sort_index().to_csv('treated/df_tblob.csv', index=False)
df_vader.sort_index().to_csv('treated/df_vader.csv', index=False)
df_bert.sort_index().to_csv('treated/df_bert.csv', index=False)